# *Libra* - Your Database Manager's Favorite Database Manager
> Brady Spears, Los Alamos National Laboratory

---

## License Information
Copyright 2024

Triad National Security, LLC. All rights reserved.
This program was produced under U.S. Government contract 89233218CNA000001 for 
Los Alamos National Laboratory (LANL), which is operated by Triad National 
Security, LLC for the U.S. Department of Energy/National Nuclear Security 
Administration. All rights in the program are reserved by Triad National 
Security, LLC, and the U.S. Department of Energy/National Nuclear Security 
Administration. The Government is granted for itself and others acting on its 
behalf a nonexclusive, paid-up, irrevocable worldwide license in this material 
to reproduce, prepare derivative works, distribute copies to the public, 
perform publicly and display publicly, and to permit others to do so.
 
Permission is hereby granted, free of charge, to any person obtaining a copy of 
this software and associated documentation files (the "Software"), to deal in 
the Software without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies 
of the Software, and to permit persons to whom the Software is furnished to do 
so, subject to the following conditions:
 
The above copyright notice and this permission notice shall be included in all 
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR 
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, 
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE 
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER 
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, 
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE 
SOFTWARE.

---

## About *Libra*
`Libra` is an open-source Python package for managing object-relation-mapped 
(ORM) instances (A.K.A. "Models") across various schemas and various relational 
database backends. `Libra` was developed by Brady Spears with 
contirubtions from Christine Gammans and Richard Alfaro-Diaz at Los Alamos 
National Laboratory.

---

## About this Notebook
This notebook demonstrates some standard use cases of `Libra`, including 
the initialization of a `Schema` object and the loading and writing of 
schema defintions to/from various built-in formats. More advanced funcitonality 
and customization options are also touched on in the latter pages of the 
notebook, but for more complete examples, please see the more focused examples 
located in the [docs/examples](docs/examples) directory.

---

## Declaring and Adding Models to a `Schema` object

In [8]:
from datetime import datetime, timezone

from sqlalchemy import Column
from sqlalchemy import DateTime, Float, Integer, String

from libra import Schema

# Initialization only needs the Schema name
kbcore = Schema('NNSA KB Core')

@kbcore.add_model
class affiliation:
    net     = Column(String(8), default = '-')
    sta     = Column(String(6), default = '-')
    time    = Column(Float(precision = 53), default = -9999999999.999)
    endtime = Column(Float(precision = 53), default = 9999999999.999)
    lddate  = Column(DateTime, nullable = False, onupdate = datetime.now(timezone.utc),
                default = datetime.now(timezone.utc))

    pk = ['net', 'sta', 'time']

Establish a connection to the database, assign a table name to a child object 
of the model, and create the table in the database, if it doesn't already exist

**CREATE**

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

connection_str = 'sqlite:///data/libra_intro.db' # In-Memory DB

engine = create_engine(connection_str)
session = Session(engine)

class Affiliation(kbcore.affiliation):
    __tablename__ = 'intro_affiliation'

Affiliation.__table__.drop(engine, checkfirst = True) # Drop table if already there

kbcore.base.metadata.create_all(engine) # Create Table

Add data to the table and commit the changes

**UPDATE**

In [10]:
# Use Positional Instantiation
session.add(Affiliation('USGS', 'ANMO', 144547200., 9999999999.999, datetime.now(timezone.utc))) # Albuquerque, NM
session.add(Affiliation('USGS', 'EDIT', 820454400., 9999999999.999, datetime.now(timezone.utc))) # Edith, TN
session.add(Affiliation('USGS', 'HAV', -552355200., 9999999999.999, datetime.now(timezone.utc))) # Havilah, CA

# Or use Keyword Instantiation (will use default column values)
session.add(Affiliation(net = 'USGS', sta = 'SGM', time = 94694400)) # Silver Gate, MT

session.commit()

Regurgitate some of the data that was just committed using `SQLAlchemy`, 
filtering on the 'time' column of the Affiliation table

**READ**

In [11]:
query = session.query(Affiliation).filter(Affiliation.time >= 0.)

list(query) # List out each row of the query

[Affiliation(net='USGS', sta='ANMO', time=144547200.0),
 Affiliation(net='USGS', sta='EDIT', time=820454400.0),
 Affiliation(net='USGS', sta='SGM', time=94694400.0)]

Delete a row from the table and query the whole table back

**DELETE**

In [12]:
aff_to_remove = session.query(Affiliation).filter(Affiliation.sta == 'EDIT').first()

session.delete(aff_to_remove)
session.commit()

list(session.query(Affiliation).all())

[Affiliation(net='USGS', sta='ANMO', time=144547200.0),
 Affiliation(net='USGS', sta='HAV', time=-552355200.0),
 Affiliation(net='USGS', sta='SGM', time=94694400.0)]